In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [4]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 1429}


1429

In [5]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': None, 'offset': 0, 'previous': None, 'total_count': 2422}


2422

In [6]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-02-22T20:29:52.991000Z,5805b59a-00a1-43ca-bde6-9265e3a69f87,0,,Jonise
1,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-02-22T20:29:23.791000Z,7b3ddbbf-163b-407c-8e28-c76e51cfcd5b,2,,Jacklin
2,379d9a3050314f1692130a596e5c2cb9,2volmar,2022-02-22T20:30:04.746000Z,f09e27de2f86473790965d654420773c,0,Laurent,Camille
3,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-02-22T20:27:28.324000Z,310596cc-8f04-4d12-bfb2-f3ed3c3c38ba,6,,Silvia
4,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-02-22T20:26:07.671000Z,793db6f5-18d3-4ea9-b178-a9115f90f39f,0,Batilus,Ymène
...,...,...,...,...,...,...,...
1327,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
1328,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
1329,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
1330,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


In [7]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect


,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
2417,bfe6fcc7ba4f4c73899c3641fe40d905,9edfab94-d4da-45e5-a15c-e82d8e688d2a,8,1,non,non,7b3ddbbf-163b-407c-8e28-c76e51cfcd5b,child
2418,bfe6fcc7ba4f4c73899c3641fe40d905,e27b6410-5ec2-4831-aaed-b8be8175485c,2,1,non,non,7b3ddbbf-163b-407c-8e28-c76e51cfcd5b,child
2419,bfe6fcc7ba4f4c73899c3641fe40d905,7459552f-759e-48fb-a95e-a0609d75cfe8,2,2,non,non,ceb954e6-7f39-4d1e-bd09-de5c49f226b5,child
2420,bfe6fcc7ba4f4c73899c3641fe40d905,01ef9ced-9f50-4f1a-a9f9-7c54e53ee6ed,11,1,non,non,ceb954e6-7f39-4d1e-bd09-de5c49f226b5,child
